# Test 4: Mass Quantities
Just test my mass quantity functions by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

### 2-1. Check the number of events for each branch
Skip this step, since I already checked many times.

### 2-2. Define mass quantities of 2 objects

In [3]:
# 2-1. Mass quantities of two objects
# A. Invariant mass M
def M(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)


# B. Transverse mass MT
def MT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    ET1, ET2 = np.sqrt(m1**2 + pt1**2), np.sqrt(m2**2 + pt2**2)
    return np.sqrt((ET1+ET2)**2 - (px1+px2)**2 - (py1+py2)**2)


# C. Transverse mass mT
# * mT is invariant under Lorentz boost along the z direction.
def mT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (pz1+pz2)**2)


# D. Transverse energy ET
def ET(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    m12 = np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)
    return np.sqrt(m12**2 + (px1+px2)**2 + (py1+py2)**2)

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels
Skip this step

## 4. Jet Clustering

### 4-1. Select stable final state particles without/with filtering out dark sector

In [4]:
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [5]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v3 = myAnal_v3.jetClustering_v1(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering_v1(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done new version


## 5. Analyze the Jets in the Truth Level

### 5-1. Preselection version 1

In [6]:
presel_bef, presel_pt, presel_pt_eta, presel_idx = myAnal_v3.preselection_v1(PseudoJet_filterDM_v3, pT_min=20, eta_max=2.5)

19373 events before preselection
19373 events after pT preselection
19373 events after pT & eta preselections
--------------------------------------------------------------------------------
0 events without PseudoJet before preselection
357 events without PseudoJet after pT preselection
519 events without PseudoJet after pT & eta preselections


## 6. Mass Quantities
```{admonition} Note
Basic background:
```
For a object, we can define the state
\begin{align}
    p^\mu = (E,\ p_x,\ p_y,\ p_z)
\end{align}
By convention $c = 1$, the mass invariant is
\begin{align}
    p_\mu p^\mu = E^2 - \vec{p}^2 = m^2
\end{align}
Hadron collider experimentalists prefer linear momentum $\vec{p}$ in terms of collider coordinate $(p_T, y(\theta), \phi)$ or $(p_T, \eta, \phi)$, the state of a object is given by
\begin{align}
    p^\mu = (m,\ p_T,\ y,\ \phi)\ \mathrm{or}\ (m,\ p_T,\ \eta,\ \phi)
\end{align}
where $p_T = |\vec{p}_T|$.
The rapidity $y$ and the pseudorapidity $\eta$ when $p \gg m$ are defined by
\begin{align}
    y = \frac{1}{2}\ln\left(\frac{E + p_z}{E - p_z}\right) \approx -\ln\left[ \tan{\left(\frac{\theta}{2}\right)} \right] \equiv \eta
\end{align}
where $\cos{\theta} = p_z/p$ and values of $\eta$ between $(-\infty,\ \infty)$.
The transformations between Cartesian and collider coordinate are
- For the rapidity $y$,
    \begin{align}
        p^\mu = (E,\ p_x,\ p_y,\ p_z) = (m_T\cosh{y},\ p_x,\ p_y,\ m_T\sinh{y})
    \end{align}
- For the pseudorapidity $\eta$ `(I use this transformation)`,
    $$
        p^\mu = (E,\ p_x,\ p_y,\ p_z) = (m_T\cosh{\eta},\ p_T\cos{\phi},\ p_T\sin{\phi},\ m_T\sinh{\eta})
    $$
where $m_T$, conventionally called the `transverse mass`, is defined by
    $$
        m_T^2 = m^2 + p_x^2 + p_y^2 = E^2 - p_z^2 = E_T
    $$
It is also denoted `transverse energy` $E_T$.
But sometimes, transverse energy vector $\vec{E}_T$ is defined by
    $$
        \vec{E}_T = E\frac{\vec{p}_T}{|\vec{p}|} = \frac{E}{\sqrt{E^2 - m^2}}\vec{p}_T
    $$
It is easy to see that $E_T = p_T = m_T$ for massless particle ($m = 0$).
From the definition one can obtin the identities
    $$
        \sinh{\eta} = \cot{\theta},\quad \cosh{\eta} = 1/\sin{\theta},\quad \tanh{\eta} = \cos{\theta}
    $$
The invariant mass $M$ of the two-object system can be written in terms of these variables as
\begin{align}
    M^2 &= \left[E_1 + E_2\right]^2 - \left[\vec{p}_1 + \vec{p}_2\right]^2 \\
    &= m_1^2 + m_2^2 + 2\left[E_T(1)E_T(2)\cosh{\Delta y} - \vec{p}_T(1)\cdot\vec{p}_T(2)\right]
\end{align}
where
    $$
        E_T^2(i) = \left| \vec{p}_T(i) \right|^2 + m_i^2
    $$
But $m_T$ is not a well quantity when final state is semi-visible (or semi-invisible) but is only indicated by missing transverse energy.
Consider a single heavy particle of mass $M$ produced in association with visible particles.
The mass of the parent particle can be constrained with the quantity $M_T$ defined by
\begin{align}
    M_T^2 &\equiv \left[E_T(1) + E_T(2)\right]^2 - \left[\vec{p}_T(1) + \vec{p_T}(2)\right]^2 \\
    &= m_1^2 + m_2^2 + 2\left[E_T(1)E_T(2) - \vec{p}_T(1)\cdot\vec{p}_T(2)\right]
\end{align}
where
\begin{align}
    E_T^2(i) &= \left| \vec{p}_T(i) \right|^2 + m_i^2,\\
    \vec{p}_T(1) &= \vec{E}_T^\mathrm{miss}
\end{align}
The distribution of event $M_T$ values possesses an end-point at $M_T^\mathrm{max} = M$.


##### References
- [[1] PDG: Kinematics (rev.)](https://pdg.lbl.gov/2021/web/viewer.html?file=../reviews/rpp2021-rev-kinematics.pdf)
- [[2] WIKI: Transverse mass](https://en.wikipedia.org/wiki/Transverse_mass)

### 6-1. Three objects

#### A. Invariant mass $M(1, 2, 3)$

In [7]:
for i in range(21,24):
    print("event {}".format(i))
    j1 = PseudoJet_filterDM_v3[i][0]
    j2 = PseudoJet_filterDM_v3[i][1]
    j3 = PseudoJet_filterDM_v3[i][2]
    M_jj = M(j1.mass, j1.pt, j1.eta, j1.phi,
             j2.mass, j2.pt, j2.eta, j2.phi)
    M_jj_new_1 = myAnal_v3.M_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                 j2.mass, j2.pt, j2.eta, j2.phi,
                                 0, 0, 0, 0)
    M_jj_new_2 = myAnal_v3.M_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                 0, 0, 0, 0,
                                 j2.mass, j2.pt, j2.eta, j2.phi)
    M_jjj_new = myAnal_v3.M_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                j2.mass, j2.pt, j2.eta, j2.phi,
                                j3.mass, j3.pt, j3.eta, j3.phi,)
    sum_1 = np.sum(M_jj_new_1 - M_jj)
    sum_2 = np.sum(M_jj_new_2 - M_jj)
    print("M(jj) old   = {}".format(M_jj))
    print("M(jj) new 1 = {}".format(M_jj_new_1), sum_1)
    print("M(jj) new 2 = {}".format(M_jj_new_2), sum_2)
    print("M(jjj) new  = {}".format(M_jjj_new))
    print('-'*80)

event 21
M(jj) old   = 466.20957770181013
M(jj) new 1 = 466.20957770181013 0.0
M(jj) new 2 = 466.20957770181013 0.0
M(jjj) new  = 784.1681298115187
--------------------------------------------------------------------------------
event 22
M(jj) old   = 45.351895655264975
M(jj) new 1 = 45.351895655264975 0.0
M(jj) new 2 = 45.351895655264975 0.0
M(jjj) new  = 149.70240792445588
--------------------------------------------------------------------------------
event 23
M(jj) old   = 1151.0740390946896
M(jj) new 1 = 1151.0740390946896 0.0
M(jj) new 2 = 1151.0740390946896 0.0
M(jjj) new  = 1283.9725242850795
--------------------------------------------------------------------------------


##### Conclusion: The function of invariant mass $M$ of 3 objects is OK!

#### B. Transverse mass $M_T(1, 2, 3)$

In [8]:
for i in range(21,24):
    print("event {}".format(i))
    j1 = PseudoJet_filterDM_v3[i][0]
    j2 = PseudoJet_filterDM_v3[i][1]
    j3 = PseudoJet_filterDM_v3[i][2]
    MT_jj = MT(j1.mass, j1.pt, j1.eta, j1.phi,
               j2.mass, j2.pt, j2.eta, j2.phi)
    MT_jj_new_1 = myAnal_v3.MT_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                   j2.mass, j2.pt, j2.eta, j2.phi,
                                   0, 0, 0, 0)
    MT_jj_new_2 = myAnal_v3.MT_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                   0, 0, 0, 0,
                                   j2.mass, j2.pt, j2.eta, j2.phi)
    MT_jjj_new = myAnal_v3.MT_123(j1.mass, j1.pt, j1.eta, j1.phi,
                                  j2.mass, j2.pt, j2.eta, j2.phi,
                                  j3.mass, j3.pt, j3.eta, j3.phi,)
    sum_1 = np.sum(MT_jj_new_1 - MT_jj)
    sum_2 = np.sum(MT_jj_new_2 - MT_jj)
    print("MT(jj) old   = {}".format(MT_jj))
    print("MT(jj) new 1 = {}".format(MT_jj_new_1), sum_1)
    print("MT(jj) new 2 = {}".format(MT_jj_new_2), sum_2)
    print("MT(jjj) new  = {}".format(MT_jjj_new))
    print('-'*80)

event 21
MT(jj) old   = 394.84919666907587
MT(jj) new 1 = 394.84919666907587 0.0
MT(jj) new 2 = 394.84919666907587 0.0
MT(jjj) new  = 628.8648625746458
--------------------------------------------------------------------------------
event 22
MT(jj) old   = 45.351895193886676
MT(jj) new 1 = 45.351895193886676 0.0
MT(jj) new 2 = 45.351895193886676 0.0
MT(jjj) new  = 129.40856630648184
--------------------------------------------------------------------------------
event 23
MT(jj) old   = 908.1050714207223
MT(jj) new 1 = 908.1050714207223 0.0
MT(jj) new 2 = 908.1050714207223 0.0
MT(jjj) new  = 948.1533195749157
--------------------------------------------------------------------------------


##### Conclusion: The function of transverse mass $M_T$ of 3 objects is OK!

#### C. Transverse mass $m_T(1, 2, 3)$ with definition 1

In [9]:
for i in range(21,24):
    print("event {}".format(i))
    j1 = PseudoJet_filterDM_v3[i][0]
    j2 = PseudoJet_filterDM_v3[i][1]
    j3 = PseudoJet_filterDM_v3[i][2]
    mT_jj = mT(j1.mass, j1.pt, j1.eta, j1.phi,
               j2.mass, j2.pt, j2.eta, j2.phi)
    mT_jj_new_1 = myAnal_v3.mT_123_def_1(j1.mass, j1.pt, j1.eta, j1.phi,
                                         j2.mass, j2.pt, j2.eta, j2.phi,
                                         0, 0, 0, 0)
    mT_jj_new_2 = myAnal_v3.mT_123_def_1(j1.mass, j1.pt, j1.eta, j1.phi,
                                         0, 0, 0, 0,
                                         j2.mass, j2.pt, j2.eta, j2.phi)
    mT_jjj_new = myAnal_v3.mT_123_def_1(j1.mass, j1.pt, j1.eta, j1.phi,
                                        j2.mass, j2.pt, j2.eta, j2.phi,
                                        j3.mass, j3.pt, j3.eta, j3.phi,)
    sum_1 = np.sum(mT_jj_new_1 - mT_jj)
    sum_2 = np.sum(mT_jj_new_2 - mT_jj)
    print("mT(jj) old   = {}".format(mT_jj))
    print("mT(jj) new 1 = {}".format(mT_jj_new_1), sum_1)
    print("mT(jj) new 2 = {}".format(mT_jj_new_2), sum_2)
    print("mT(jjj) new  = {}".format(mT_jjj_new))
    print('-'*80)

event 21
mT(jj) old   = 899.9222928467409
mT(jj) new 1 = 899.9222928467409 0.0
mT(jj) new 2 = 899.9222928467409 0.0
mT(jjj) new  = 1061.0420901057623
--------------------------------------------------------------------------------
event 22
mT(jj) old   = 163.92791393503617
mT(jj) new 1 = 163.92791393503617 0.0
mT(jj) new 2 = 163.92791393503617 0.0
mT(jjj) new  = 202.19992339522818
--------------------------------------------------------------------------------
event 23
mT(jj) old   = 1153.6319519024476
mT(jj) new 1 = 1153.6319519024476 0.0
mT(jj) new 2 = 1153.6319519024476 0.0
mT(jjj) new  = 1289.041909267317
--------------------------------------------------------------------------------


##### Conclusion: The function of transverse mass $m_T$ of 3 objects is OK!

#### D. Transverse mass $m_T(1, 2, 3)$ with definition 2

In [10]:
for i in range(21,24):
    print("event {}".format(i))
    j1 = PseudoJet_filterDM_v3[i][0]
    j2 = PseudoJet_filterDM_v3[i][1]
    j3 = PseudoJet_filterDM_v3[i][2]
    ET_jj = ET(j1.mass, j1.pt, j1.eta, j1.phi,
               j2.mass, j2.pt, j2.eta, j2.phi)
    mT_jj_new_1 = myAnal_v3.mT_123_def_2(j1.mass, j1.pt, j1.eta, j1.phi,
                                         j2.mass, j2.pt, j2.eta, j2.phi,
                                         0, 0, 0, 0)
    mT_jj_new_2 = myAnal_v3.mT_123_def_2(j1.mass, j1.pt, j1.eta, j1.phi,
                                         0, 0, 0, 0,
                                         j2.mass, j2.pt, j2.eta, j2.phi)
    mT_jjj_new = myAnal_v3.mT_123_def_2(j1.mass, j1.pt, j1.eta, j1.phi,
                                        j2.mass, j2.pt, j2.eta, j2.phi,
                                        j3.mass, j3.pt, j3.eta, j3.phi,)
    sum_1 = np.sum(mT_jj_new_1 - ET_jj)
    sum_2 = np.sum(mT_jj_new_2 - ET_jj)
    print("ET(jj) old   = {}".format(ET_jj))
    print("mT(jj) new 1 = {}".format(mT_jj_new_1), sum_1)
    print("mT(jj) new 2 = {}".format(mT_jj_new_2), sum_2)
    print("mT(jjj) new  = {}".format(mT_jjj_new))
    print('-'*80)

event 21
ET(jj) old   = 899.9222928467409
mT(jj) new 1 = 899.9222928467409 0.0
mT(jj) new 2 = 899.9222928467409 0.0
mT(jjj) new  = 1061.0420901057623
--------------------------------------------------------------------------------
event 22
ET(jj) old   = 163.92791393503623
mT(jj) new 1 = 163.92791393503623 0.0
mT(jj) new 2 = 163.92791393503623 0.0
mT(jjj) new  = 202.1999233952283
--------------------------------------------------------------------------------
event 23
ET(jj) old   = 1153.6319519024478
mT(jj) new 1 = 1153.6319519024478 0.0
mT(jj) new 2 = 1153.6319519024478 0.0
mT(jjj) new  = 1289.041909267317
--------------------------------------------------------------------------------


### 6-2. General functions of mass quantities

### Test general function

In [19]:
PseudoJet_filterDM_v3[23][:4]

[PseudoJet(pt=492.576, eta=-0.905, phi=-1.942, mass=47.887),
 PseudoJet(pt=415.898, eta=0.528, phi=1.208, mass=21.372),
 PseudoJet(pt=42.026, eta=-2.093, phi=-2.574, mass=11.838),
 PseudoJet(pt=15.899, eta=-0.073, phi=-0.441, mass=3.713)]

In [20]:
presel_pt_eta[23]

array([(492.57649468, -0.90459915, -1.94196123, 47.88720232),
       (415.89764971,  0.52782149,  1.20835548, 21.37220707),
       ( 42.02645083, -2.09324215, -2.5737621 , 11.83759252)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [21]:
presel_pt_eta[23]['pT']

array([492.57649468, 415.89764971,  42.02645083])

In [33]:
np.array(presel_pt_eta[23]['pT']) - presel_pt_eta[23]['pT']

array([0., 0., 0.])

In [ ]:
def f_M(a, b, c, d):
    print()

#### test

### 6-4. Test interesting method!!

In [47]:
def f(a, b, c, d):
    # a=mass, b=pt, c=eta, d=phi
    a, b = np.array(a), np.array(b)
    c, d = np.array(c), np.array(d)
    for i in range(len(a)):
        arr_px = b * d
        arr_py = b * d
        arr_pz = np.sqrt(a**2 + b**2) * c
        arr_e = np.sqrt(a**2 + arr_px**2 + arr_py**2 + arr_pz**2)
        print(i)
    
    return arr_e, arr_px, arr_py, arr_pz


ee, pxx, pyy, pzz = f([1, 2, 3], [4, 5, 6], [1, 3, 5], [2, 4, 6])
ee

0
1
2


array([12.08304597, 32.63433774, 61.04096985])

In [46]:
print(f([1, 2, 3], [4, 5, 6], [1, 3, 5], [2, 4, 6])[2] * 2)
print('-'*80)
f([1, 2, 3], [4, 5, 6], [1, 3, 5], [2, 4, 6])

0
1
2
[ 2  6 10]
--------------------------------------------------------------------------------
0
1
2


(array([2, 4, 6]), array([4, 5, 6]), array([1, 3, 5]), array([2, 4, 6]))

In [36]:
a = [1, 2, 3, 4, 5]
a[:3]

[1, 2, 3]

In [41]:
a = np.array([1, 2, 3, 4, 5, 9])
np.sqrt(a)

array([1.        , 1.41421356, 1.73205081, 2.        , 2.23606798,
       3.        ])

In [42]:
a**2

array([ 1,  4,  9, 16, 25, 81])